In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для тайм-менеджмента
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from tqdm import tqdm
from time import sleep
from webdriver_manager.firefox import GeckoDriverManager
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.by import By
from sys import platform
import nltk
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from random import SystemRandom
from selenium import webdriver
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from string import punctuation
nltk.download("stopwords")
import datetime

[nltk_data] Downloading package stopwords to /home/victor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
MonthDict={ "янв" : 1,
      "февр": 2,
       "мар": 3,
       "апр": 4,
       "мая": 5,
       "июн": 6,
       "июл": 7,
       "авг": 8,
       "сент": 9,
       "окт": 10,
       "нояб": 11,
       "дек": 12
}

rq = pd.read_excel('smantika_remonty.xlsx', header=None)
rq = rq.rename(columns={0:'req'})
df = pd.DataFrame()

In [15]:

def init():
    options = FirefoxOptions()
   
    #opts.add_argument("--headless")

    #options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    #options.add_argument('--MOZ_LOG_FILE=/root/firefox.log')
    #options.add_argument("start-maximized")
    #options.add_argument("disable-infobars")
    #options.add_argument("--disable-extensions")
    #options.add_argument("--no-sandbox")
    #options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox( firefox_options=options, executable_path=GeckoDriverManager().install())
    #driver.get(page_link)
    return driver
def search(request):
    driver.get('https://www.youtube.com/results?search_query={}'.format(str(request)))
    time.sleep(3)
    filter_class = '/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a'
    filter_bar = driver.find_element_by_xpath(filter_class)
    filter_bar.click()
    
    time.sleep(3)
    chanel_filter_cl= '/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[2]/a/div/yt-formatted-string'
    chanel_filter = driver.find_element_by_xpath(chanel_filter_cl)
    chanel_filter.click()

    time.sleep(3)
    
    for i in (range(15)):
        driver.execute_script("window.scrollTo(0, 100000)") 
        time.sleep(2)
def load_channels(df):
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    res = soup.findAll('div', attrs = {'id':'info-section'})
    names=[]
    links=[]
    subs = []
    for i in range(len(res)):
        link = 'https://www.youtube.com' + res[i].findAll('a')[0]['href']
        name = res[i].findAll('div', attrs = {'class':'hidden style-scope paper-tooltip'})[0].text.replace('\n',
                                                                                                           '').strip()
        subscribers = res[i].findAll('span', attrs = {'id':'subscribers'})[0].text.replace('\xa0', ' ').replace('подписчиков', '').strip()
        if 'тыс.' in subscribers:
            subscribers = subscribers.replace('тыс.', '').strip()+'000'
            if ',' not in subscribers:
                subscribers = int(subscribers)
            if ',' in str(subscribers):
                subscribers = int(subscribers.replace(',','')[:-1])
        elif 'млн' in subscribers:
            subscribers = subscribers.replace('млн', '').strip()+'000000'
            if ',' not in subscribers:
                subscribers = int(subscribers)
            if ',' in str(subscribers):
                subscribers = int(subscribers.replace(',','')[:-1])
        else:
            subscribers = 0

        names.append(name)
        links.append(link)
        subs.append(subscribers)
    df = df.append(pd.DataFrame({'name': names,
         'link': links,
         'subs': subs}))
    df.index = np.arange(len(df))
    df = df.drop_duplicates()
    return df

def go_to_ch(link):
    
    driver.get(link)
    
def get_ch_info():
    info_path1 = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/div[3]/ytd-c4-tabbed-header-renderer/app-header-layout/div/app-header/div[2]/app-toolbar/div/div/paper-tabs/div/div/paper-tab[6]'
    info_path2 = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/div[3]/ytd-c4-tabbed-header-renderer/app-header-layout/div/app-header/div[2]/app-toolbar/div/div/paper-tabs/div/div/paper-tab[5]/div'
    try:
        info_button = driver.find_element_by_xpath(info_path1)
        info_button.click()
    except:
        info_button = driver.find_element_by_xpath(info_path2)
        info_button.click()
    
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    try:
        date = soup.findAll('span', attrs = {'class': 'style-scope yt-formatted-string'})[-1].text
        date = date[:-2]
        date = date.replace("Дата регистрации:", '')
        date = date.replace('.', '')
        date = date.replace(' ', '')
        if date[:2].isnumeric():
            day = date[:2]
            year = date[-4:]
            month = str(MonthDict[date[2:-4]])
            date = day+'/'+month+'/'+year
            date = datetime.datetime.strptime(date, '%d/%m/%Y')
        elif not date[1].isnumeric():
            day = date[:1]
            year = date[-4:]
            month = str(MonthDict[date[1:-4]])
            date = day+'/'+month+'/'+year
            date = datetime.datetime.strptime(date, '%d/%m/%Y')
    except:
        date = 0
        print('no date')
        
    
    try:
        views = soup.findAll('yt-formatted-string', attrs = {'class': 'style-scope ytd-channel-about-metadata-renderer'})[-1].text
        views = views.replace('\xa0','')
        views = views.replace('просмотров','')
        views = views.replace('просмотра','')
        views = views.replace('просмотр','')
        views = int(views.strip())
    except:
        views = 0
        print('no views')
    return views, date
def get_desc():
    info_path1 = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/div[3]/ytd-c4-tabbed-header-renderer/app-header-layout/div/app-header/div[2]/app-toolbar/div/div/paper-tabs/div/div/paper-tab[6]'
    info_path2 = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/div[3]/ytd-c4-tabbed-header-renderer/app-header-layout/div/app-header/div[2]/app-toolbar/div/div/paper-tabs/div/div/paper-tab[5]/div'
    try:
        info_button = driver.find_element_by_xpath(info_path1)
        info_button.click()
    except:
        info_button = driver.find_element_by_xpath(info_path2)
        info_button.click()
    
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    
    try:
        desc = soup.findAll('div', attrs={'id': 'description-container'})[0].text.replace('\n', ' ').strip()
    except:
        print('no desc')
        desc = 'no desc'
    return desc

In [4]:
driver = init()

[WDM] - Getting latest mozilla release info for v0.28.0
[WDM] - Trying to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.28.0/geckodriver-v0.28.0-linux64.tar.gz
[WDM] - Driver has been saved in cache [/home/victor/.wdm/drivers/geckodriver/linux64/v0.28.0]
/home/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of firefox_options
  


In [5]:
df = pd.read_csv('links.csv')
df

,Unnamed: 0,name,link,subs
0,0,Ремонт квартир под ключ. Юрий Литвин,https://www.youtube.com/channel/UCSUuUsjKg85OH...,0
1,1,Александр Комиссаров Ремонт Квартир в Москве,https://www.youtube.com/channel/UCGgC-Nd-DNmaR...,27400
2,2,Forcemontage - Ремонт Квартир Ремонт домов,https://www.youtube.com/user/forcemontage,138000
3,3,Ремонт квартир в Москве,https://www.youtube.com/channel/UCq5_ndCtTwI4m...,0
4,4,ГРУППА КОМПАНИЙ РЕМОНТ ПОД КЛЮЧ,https://www.youtube.com/channel/UCPIoogs508F3I...,0
...,...,...,...,...
5137,5218,SanKraft Березовский Станислав,https://www.youtube.com/channel/UC01wSD2-S0m3q...,0
5138,5220,poselok-britanika.ru,https://www.youtube.com/channel/UCkRoocKS8PT76...,0
5139,5221,GOR STROY,https://www.youtube.com/channel/UCv-Ks0G81aQJi...,0
5140,5222,Александр Архипов,https://www.youtube.com/channel/UC6u5Kliw-Ll-x...,0


In [7]:
go_to_ch(df['link'][1])

In [8]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [14]:
desc = soup.findAll('div', attrs={'id': 'description-container'})[0].text.replace('\n', ' ').strip()
desc

'Описание Всех приветствую ! Наш канал о строительстве и ремонте квартир в Москве, но не только квартир домов и коттеджей, а и малогабаритных квартир в Москве и Московской области. Мы рады вам предложить наши услуги, которые избавят Вас от головной боли которая в основном возникает при ремонте квартир, мы честны и открыты для всех желающих! Для многих ремонт это целая жизнь! Мы эту жизнь проживем за Вас ...  Вы просто отдыхайте в момент нашей работы, а мы все сделаем за Вас!  Итак обращайтесь, подскажем, проконсультируем, поможем! Наши контакты Наш сайт: https://moskovsky-remont.ru/ ВК: https://vk.com/expresservis Инстаграм: https://www.instagram.com/moskva_remont_kvartir_/?hl=ru Почта: express_servis1@mail.ru'

In [6]:
for request in tqdm(rq['req']):
    search(request)
    df = load_channels(df)

100%|██████████| 69/69 [47:59<00:00, 41.73s/it]


In [16]:
views = []
date = []
for link in tqdm(df['link']):
    go_to_ch(link)
    v,d = get_ch_info()
    views.append(v)
    date.append(d)
    
df['views'] = views
df['date'] = date

df.to_csv('full_info.csv')




  0%|          | 0/5242 [00:00<?, ?it/s]


  0%|          | 1/5242 [00:04<5:52:48,  4.04s/it]


  0%|          | 2/5242 [00:08<5:58:38,  4.11s/it]


  0%|          | 3/5242 [00:12<5:58:39,  4.11s/it]


  0%|          | 4/5242 [00:16<6:04:57,  4.18s/it]


  0%|          | 5/5242 [00:20<6:03:48,  4.17s/it]


  0%|          | 6/5242 [00:24<5:42:51,  3.93s/it]


  0%|          | 7/5242 [00:27<5:28:37,  3.77s/it]


  0%|          | 8/5242 [00:31<5:27:36,  3.76s/it]


  0%|          | 9/5242 [00:35<5:26:11,  3.74s/it]


  0%|          | 10/5242 [00:38<5:22:30,  3.70s/it]


  0%|          | 11/5242 [00:42<5:37:23,  3.87s/it]


  0%|          | 12/5242 [00:46<5:25:50,  3.74s/it]


  0%|          | 13/5242 [00:50<5:25:18,  3.73s/it]


  0%|          | 14/5242 [00:53<5:17:07,  3.64s/it]


  0%|          | 15/5242 [00:57<5:20:04,  3.67s/it]


  0%|          | 16/5242 [01:00<5:20:06,  3.68s/it]


  0%|          | 17/5242 [01:04<5:18:13,  3.65s/it]


  0%|          | 18/5242 [01:08<5:12:08,  3.5

no views





  0%|          | 19/5242 [01:11<5:14:30,  3.61s/it]


  0%|          | 20/5242 [01:15<5:11:48,  3.58s/it]


  0%|          | 21/5242 [01:19<5:20:36,  3.68s/it]


  0%|          | 22/5242 [01:22<5:15:59,  3.63s/it]


  0%|          | 23/5242 [01:26<5:18:57,  3.67s/it]


  0%|          | 24/5242 [01:29<5:14:44,  3.62s/it]


  0%|          | 25/5242 [01:33<5:08:57,  3.55s/it]


  0%|          | 26/5242 [01:37<5:17:21,  3.65s/it]


  1%|          | 27/5242 [01:41<5:33:46,  3.84s/it]


  1%|          | 28/5242 [01:45<5:33:46,  3.84s/it]


  1%|          | 29/5242 [01:48<5:28:00,  3.78s/it]


  1%|          | 30/5242 [01:52<5:31:15,  3.81s/it]


  1%|          | 31/5242 [01:56<5:27:32,  3.77s/it]


  1%|          | 32/5242 [02:01<6:05:42,  4.21s/it]


  1%|          | 33/5242 [02:05<5:56:15,  4.10s/it]


  1%|          | 34/5242 [02:12<7:03:53,  4.88s/it]


  1%|          | 35/5242 [02:15<6:30:49,  4.50s/it]


  1%|          | 36/5242 [02:19<6:17:05,  4.35s/it]


  1%|          | 37/5242 

  5%|▌         | 276/5242 [19:33<5:32:09,  4.01s/it]


  5%|▌         | 277/5242 [19:37<5:36:08,  4.06s/it]


  5%|▌         | 278/5242 [19:41<5:24:12,  3.92s/it]


  5%|▌         | 279/5242 [19:45<5:18:43,  3.85s/it]


  5%|▌         | 280/5242 [19:48<5:13:46,  3.79s/it]


  5%|▌         | 281/5242 [19:53<5:29:49,  3.99s/it]


  5%|▌         | 282/5242 [19:57<5:39:50,  4.11s/it]


  5%|▌         | 283/5242 [20:01<5:31:12,  4.01s/it]


  5%|▌         | 284/5242 [20:05<5:24:26,  3.93s/it]


  5%|▌         | 285/5242 [20:08<5:19:47,  3.87s/it]


  5%|▌         | 286/5242 [20:12<5:20:51,  3.88s/it]


  5%|▌         | 287/5242 [20:17<5:48:28,  4.22s/it]


  5%|▌         | 288/5242 [20:22<6:09:47,  4.48s/it]


  6%|▌         | 289/5242 [20:26<5:58:09,  4.34s/it]


  6%|▌         | 290/5242 [20:32<6:41:23,  4.86s/it]


  6%|▌         | 291/5242 [20:36<6:13:09,  4.52s/it]


  6%|▌         | 292/5242 [20:40<5:48:20,  4.22s/it]


  6%|▌         | 293/5242 [20:44<5:56:43,  4.32s/it]


  6%|▌    

no views





  8%|▊         | 411/5242 [28:44<5:08:31,  3.83s/it]


  8%|▊         | 412/5242 [28:48<5:11:07,  3.86s/it]


  8%|▊         | 413/5242 [28:52<5:10:58,  3.86s/it]


  8%|▊         | 414/5242 [28:56<5:11:26,  3.87s/it]


  8%|▊         | 415/5242 [29:00<5:17:52,  3.95s/it]


  8%|▊         | 416/5242 [29:04<5:29:27,  4.10s/it]


  8%|▊         | 417/5242 [29:08<5:21:40,  4.00s/it]


  8%|▊         | 418/5242 [29:11<5:11:27,  3.87s/it]


  8%|▊         | 419/5242 [29:15<5:11:53,  3.88s/it]


  8%|▊         | 420/5242 [29:19<5:10:45,  3.87s/it]


  8%|▊         | 421/5242 [29:23<5:14:01,  3.91s/it]


  8%|▊         | 422/5242 [29:28<5:23:39,  4.03s/it]


  8%|▊         | 423/5242 [29:31<5:11:29,  3.88s/it]


  8%|▊         | 424/5242 [29:35<5:17:37,  3.96s/it]


  8%|▊         | 425/5242 [29:39<5:23:14,  4.03s/it]


  8%|▊         | 426/5242 [29:43<5:22:08,  4.01s/it]


  8%|▊         | 427/5242 [29:51<6:58:49,  5.22s/it]


  8%|▊         | 428/5242 [29:55<6:16:16,  4.69s/it]


  8%|▊ 

KeyboardInterrupt: 

In [5]:
df = load_channels(df)


In [18]:
df

,name,link,subs
0,Ремонт квартир под ключ. Юрий Литвин,https://www.youtube.com/channel/UCSUuUsjKg85OH...,0
1,Александр Комиссаров Ремонт Квартир в Москве,https://www.youtube.com/channel/UCGgC-Nd-DNmaR...,27400
2,Forcemontage - Ремонт Квартир Ремонт домов,https://www.youtube.com/user/forcemontage,138000
3,Ремонт квартир в Москве,https://www.youtube.com/channel/UCq5_ndCtTwI4m...,0
4,ГРУППА КОМПАНИЙ РЕМОНТ ПОД КЛЮЧ,https://www.youtube.com/channel/UCPIoogs508F3I...,0
...,...,...,...
5218,SanKraft Березовский Станислав,https://www.youtube.com/channel/UC01wSD2-S0m3q...,0
5220,poselok-britanika.ru,https://www.youtube.com/channel/UCkRoocKS8PT76...,0
5221,GOR STROY,https://www.youtube.com/channel/UCv-Ks0G81aQJi...,0
5222,Александр Архипов,https://www.youtube.com/channel/UC6u5Kliw-Ll-x...,0


In [52]:
go_to_ch(df['link'][1])

In [53]:
get_ch_info()

(13019639, datetime.datetime(2011, 10, 19, 0, 0))

In [54]:
date = []
views = []

In [57]:
df['link'][27]

'https://www.youtube.com/channel/UCud1dd4B5AXX6kIdoB76Fnw'

In [62]:

for link in tqdm(df['link'][40:]):
    go_to_ch(link)
    v,d = get_ch_info()
    views.append(v)
    date.append(d)








  0%|          | 0/38 [00:00<?, ?it/s]






  3%|▎         | 1/38 [00:04<02:46,  4.51s/it]






  5%|▌         | 2/38 [00:08<02:40,  4.46s/it]






  8%|▊         | 3/38 [00:12<02:28,  4.25s/it]






 11%|█         | 4/38 [00:16<02:21,  4.17s/it]






 13%|█▎        | 5/38 [00:21<02:20,  4.25s/it]






 16%|█▌        | 6/38 [00:25<02:14,  4.19s/it]






 18%|█▊        | 7/38 [00:29<02:10,  4.22s/it]






 21%|██        | 8/38 [00:33<02:03,  4.12s/it]






 24%|██▎       | 9/38 [00:37<01:58,  4.08s/it]






 26%|██▋       | 10/38 [00:41<01:59,  4.26s/it]






 29%|██▉       | 11/38 [00:46<01:54,  4.24s/it]






 32%|███▏      | 12/38 [00:49<01:47,  4.12s/it]






 34%|███▍      | 13/38 [00:54<01:46,  4.28s/it]






 37%|███▋      | 14/38 [00:58<01:43,  4.30s/it]






 39%|███▉      | 15/38 [01:03<01:38,  4.30s/it]






 42%|████▏     | 16/38 [01:07<01:32,  4.23s/it]






 45%|████▍     | 17/38 [01:11<01:30,  4.30s/it]






 47%|████▋     | 18/38 [01:15<01:24,  4

In [64]:
df['views'] = views

In [65]:
df['date'] = date

In [66]:
df

,name,link,subs,views,date
0,Ремонт квартиры своими руками (Remontkv.pro),https://www.youtube.com/user/remontkvpro,527000,71959972,2013-10-17
1,Helpdroid,https://www.youtube.com/user/TheStatusvip,75900,13019639,2011-10-19
2,Ремонт квартир 2015 в Москве,https://www.youtube.com/channel/UCfA3qPFAShTfc...,0,80453,2015-05-07
3,Ремонт Без Купюр,https://www.youtube.com/channel/UC7avPVOCzu1Ff...,0,235668,2015-10-15
4,Daniil Gerasimov,https://www.youtube.com/user/ppsip,281000,43075195,2011-05-21
...,...,...,...,...,...
74,"Интерьер-дизайн, ремонт под ключ",https://www.youtube.com/channel/UCT03lELMLWe1f...,0,32912,2016-12-20
75,Компания КВАДРАТУМ - ремонт квартир в Одессе,https://www.youtube.com/channel/UCP_Ks84qCUEtD...,0,1599,2018-12-07
76,Ремонт квартир в Красноярске,https://www.youtube.com/channel/UCal6NyhLeurjO...,0,13290,2016-02-03
77,РЕМОНТ 46,https://www.youtube.com/channel/UCSCLXI3bY56VR...,0,7342,2016-03-05


In [21]:
get_ch_info()

KeyError: 'ав'

In [36]:
a = driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-browse/div[3]/ytd-c4-tabbed-header-renderer/app-header-layout/div/app-header/div[2]/app-toolbar/div/div/paper-tabs/div/div/paper-tab[5]/div')

In [37]:
a.click()

In [35]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [26]:
soup.findAll('yt-formatted-string', attrs = {'class': 'style-scope ytd-channel-about-metadata-renderer'})

[<yt-formatted-string class="style-scope ytd-channel-about-metadata-renderer" id="description" split-lines="">Здравствуйте, друзья! Это канал Алексея Земскова. Здесь вы найдёте полезные советы по ремонту: как сэкономить, как избежать ошибок, как проверить рабочих, как выбрать материалы, как сделать дизайн или ремонт своими руками.
 * * *
 Рекламой за деньги я не занимаюсь, могу помочь бесплатно. У меня есть сайт, где я бесплатно размещаю информацию о своих партнёрах. Вы тоже можете попасть в их ряды. В результате о вас узнают все подписчики Инстаграма и Ютьюба, их более 1 млн человек. Заходите на сайт ZemSkidki.ru и оставляйте заявку, её рассмотрит руководитель проекта и свяжется с вами.
 * * *
 Алексей Земсков, Земской, Земского, ремонт квартир, ремонт квартиры, alexey zemskov, лайфхаки ремонта, дизайн квартир, ремонт дома, ремонт своими руками, электрика, электромонтаж, сантехника, монтаж сантехники, пластиковые окна ПВХ, межкомнатные двери, входные двери, установк дверей, монтаж две

In [29]:
date = soup.findAll('span', attrs = {'class': 'style-scope yt-formatted-string'})[-1].text

In [30]:
date = date[:-2]
date = date.replace('.', '')
date = date.replace(' ', '')
date

'8авг2011'

In [15]:
date

datetime.datetime(2015, 12, 20, 0, 0)

In [14]:
if date[:2].isnumeric():
    day = date[:2]
    year = date[-4:]
    month = str(MonthDict[date[2:-4]])
    date = day+'/'+month+'/'+year
    date = datetime.datetime.strptime(date, '%d/%m/%Y')
elif not date[1].isnumeric():
    day = date[:1]
    year = date[-4:]
    month = str(MonthDict[date[1:-4]])
    date = day+'/'+month+'/'+year
    date = datetime.datetime.strptime(date, '%d/%m/%Y')

In [21]:
views = soup.findAll('yt-formatted-string', attrs = {'class': 'style-scope ytd-channel-about-metadata-renderer'})[-1].text
views = views.replace('\xa0','')
views = int(views.replace('просмотров','').strip())
views

'6387486'

In [25]:
soup = BeautifulSoup(driver.page_source,'html.parser')
res = soup.findAll('div', attrs = {'id':'info-section'})
names=[]
links=[]
subs = []
for i in range(len(res)):
    link = res[i].findAll('a')[0]['href']
    name = 'https://www.youtube.com' + res[i].findAll('div', attrs = {'class':'hidden style-scope paper-tooltip'})[0].text.replace('\n',
                                                                                                       '').strip()
    subscribers = res[i].findAll('span', attrs = {'id':'subscribers'})[0].text.replace('\xa0', ' ').replace('подписчиков', '').strip()
    if 'тыс.' in subscribers:
        subscribers = subscribers.replace('тыс.', '').strip()+'000'
        if ',' not in subscribers:
            subscribers = int(subscribers)
        if ',' in str(subscribers):
            subscribers = int(subscribers.replace(',','')[:-1])
    elif 'млн' in subscribers:
        subscribers = subscribers.replace('млн', '').strip()+'000000'
        if ',' not in subscribers:
            subscribers = int(subscribers)
        if ',' in str(subscribers):
            subscribers = int(subscribers.replace(',','')[:-1])
    else:
        subscribers = 0

    names.append(name)
    links.append(link)
    subs.append(subscribers)
df.append(pd.DataFrame({'name': names,
     'link': links,
     'subs': subs}))
df.index = np.arange(len(df))

In [35]:
df

,name,link,subs
0,ДИЗАЙН ИНТЕРЬЕРА,/channel/UCtORu6mNAoGAcj0rwdspJFA,403000
1,ДНЕВНИК ДИЗАЙНЕРА,/channel/UCbhIVqiD-KnowMYFsGZ1xqQ,767000
2,GEOMETRIUM — cовременный дизайн интерьера,/channel/UCssuw4KHM9I7_whvaIjyU5g,305000
3,Дизайн,/channel/UCswtUaxvXXZe3KkwMtgrj9g,29200
4,Design Prosmotr,/channel/UCwhdOXBJIuNWFXYNQid6ciw,81700
...,...,...,...
72,TaupeHOME Дизайн-бюро,/channel/UCCn0CPj2dcf37XYDh3PXuLQ,0
73,Fairy Fox Design,/channel/UCg47QOdTpGRExD-ucAkmI1Q,11400
74,Inclusive Design 24 #id24,/channel/UCweRb-Dck2CfVrkeb51JXOQ,31900
75,FV Design,/user/RckstrModStudio,47500


In [8]:
link = res[0].findAll('a')[0]['href']

In [9]:
name = res[0].findAll('div', attrs = {'class':'hidden style-scope paper-tooltip'})[0].text.replace('\n', '').strip()

In [12]:
subscribers = res[0].findAll('span', attrs = {'id':'subscribers'})[0].text.replace('\xa0', ' ').replace('подписчиков', '').strip()
if 'тыс.' in subscribers:
    subscribers = subscribers.replace('тыс.', '').strip()+'000'
    if ',' not in subscribers:
        subscribers = int(subscribers)
    if ',' in str(subscribers):
        subscribers = int(subscribers.replace(',','')[:-1])

else:
    subscribers = 0

403000

In [8]:
names=[]
links=[]
subs = []
for i in range(len(res)):
    link = res[i].findAll('a')[0]['href']
    name = res[i].findAll('div', attrs = {'class':'hidden style-scope paper-tooltip'})[0].text.replace('\n',
                                                                                                       '').strip()
    subscribers = res[i].findAll('span', attrs = {'id':'subscribers'})[0].text.replace('\xa0', ' ').replace('подписчиков', '').strip()
    if 'тыс.' in subscribers:
        subscribers = subscribers.replace('тыс.', '').strip()+'000'
        if ',' not in subscribers:
            subscribers = int(subscribers)
        if ',' in str(subscribers):
            subscribers = int(subscribers.replace(',','')[:-1])
    else:
        subscribers = 0
    
    names.append(name)
    links.append(link)
    subs.append(subscribers)

In [10]:
df = pd.DataFrame({'name': names,
             'link': links,
             'subs': subs})

In [12]:
df.append(pd.DataFrame({'name': names,
             'link': links,
             'subs': subs}))

,name,link,subs
0,ДИЗАЙН ИНТЕРЬЕРА,/channel/UCtORu6mNAoGAcj0rwdspJFA,403000
1,ДНЕВНИК ДИЗАЙНЕРА,/channel/UCbhIVqiD-KnowMYFsGZ1xqQ,767000
2,GEOMETRIUM — cовременный дизайн интерьера,/channel/UCssuw4KHM9I7_whvaIjyU5g,305000
3,Дизайн,/channel/UCswtUaxvXXZe3KkwMtgrj9g,29200
4,Design Prosmotr,/channel/UCwhdOXBJIuNWFXYNQid6ciw,81700
...,...,...,...
71,Ландшафтный дизайн Кости Юдина,/user/konstantinudin,16500
72,Дизайн-бюро Виктории Файнблат,/channel/UCHKEYv9_y8rpd-5CDbeIgNw,32400
73,Ландшафтный Дизайн,/user/landshort,26600
74,Graphic Design Studio,/channel/UCPO4KsEEH0uJx6kJ2Dudu-A,12900
